In [ ]:
import pandas as pd
import boto3
import json
import configparser
# Create an instance of ConfigParser
config = configparser.ConfigParser()
# Read the configuration file
config.read_file(open('dwh.cfg'))

# Retrieve the values from the configuration file using the specified keys
KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

# Create a pandas DataFrame to display the parameters and their corresponding values
pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

In [ ]:
import boto3

# Create an EC2 resource object
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

# Create an S3 resource object
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

# Create an IAM client object
iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

# Create a Redshift client object
redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

In [ ]:
# Retrieve the bucket named 'udacity-dend'
bucket=s3.Bucket('udacity-dend')

# Get a list of log data file names from the bucket
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix='log-data')]

# Display the first 10 log data file names
log_data_files[:10]
# Get a list of song data file names from the bucket
song_data_files = [filename.key for filename in bucket.objects.filter(Prefix='song-data/A/A')]
song_data_files[:10]

In [ ]:
try:
# Create an IAM role with the specified parameters
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
# Attach the Amazon S3 read-only access policy to the IAM role
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']
# Retrieve the ARN of the IAM role
RoleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

In [ ]:
try:
    # Create a Redshift cluster with the specified parameters
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[RoleArn]  
    )
except Exception as e:
    print(e)

In [ ]:
def prettyRedshiftProps(props):
     """
    Formats and displays Redshift cluster properties in a pandas DataFrame.
    
    Args:
        props (dict): Dictionary containing Redshift cluster properties.
        
    Returns:
        pandas.DataFrame: Formatted DataFrame with selected cluster properties.
    """
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

# Retrieve the properties of the Redshift cluster
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

# Display the formatted cluster properties
prettyRedshiftProps(myClusterProps)

In [ ]:
# Retrieve the endpoint and role ARN from cluster properties
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
try:
    # Retrieve the VPC and default security group
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    # Authorize ingress for the default security group
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

In [ ]:
# Load SQL extension
%load_ext sql

In [ ]:
# Create the connection string
import os 
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)

# Connect to the Redshift cluster
%sql $conn_string

In [ ]:
# Execute the series of SQL queries
%%sql
count_user << select count(*) as total_user from dim_user

In [ ]:
%%sql
count_song << select count(*) as total_song from dim_song

In [ ]:
%%sql
count_artist << select count(*) as total_artist from dim_artist

In [ ]:
%%sql
count_time << select count(*) as total_time from dim_time

In [ ]:
%%sql
count_songplay << select count(*) as total_songplay from fact_songplay

In [ ]:
# Import the tabulate library
from tabulate import tabulate

# Format and display the query results
data = [
    ["Total User", count_user],
    ["Total Song", count_song],
    ["Total Artist", count_artist],
    ["Total Time", count_time],
    ["Total Songplay", count_songplay]
]

headers = ["Category", "Count"]
table = tabulate(data, headers, tablefmt="fancy_grid")
print(table)

In [ ]:
%%sql
-- Finding the most active users based on the number of songs played
SELECT first_name,last_name, COUNT(songplay_id) AS songs_played
FROM fact_songplay f
JOIN dim_user d ON f.user_id = d.user_id
GROUP BY first_name, last_name
ORDER BY songs_played DESC
LIMIT 10;

In [ ]:
%%sql
-- Finding the most popular songs based on the number of times they have been played
SELECT title as song_name, name as singer_name, COUNT(songplay_id) AS play_count
FROM fact_songplay f
JOIN dim_song d ON f.song_id = d.song_id
JOIN dim_artist a ON d.artist_id = a.artist_id
GROUP BY title,name
ORDER BY play_count DESC
LIMIT 10;

In [ ]:
# Delete the Redshift cluster
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

# Retrieve the updated cluster properties
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

# Display the updated cluster properties
prettyRedshiftProps(myClusterProps)